# Website A/B Testing - Lab

## Introduction

In this lab, you'll get another chance to practice your skills at conducting a full A/B test analysis. It will also be a chance to practice your data exploration and processing skills! The scenario you'll be investigating is data collected from the homepage of a music app page for audacity.

## Objectives

You will be able to:
* Analyze the data from a website A/B test to draw relevant conclusions
* Explore and analyze web action data

## Exploratory Analysis

Start by loading in the dataset stored in the file 'homepage_actions.csv'. Then conduct an exploratory analysis to get familiar with the data.

> Hints:
    * Start investigating the id column:
        * How many viewers also clicked?
        * Are there any anomalies with the data; did anyone click who didn't view?
        * Is there any overlap between the control and experiment groups? 
            * If so, how do you plan to account for this in your experimental design?

In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
df = pd.read_csv('homepage_actions.csv')
print(df.head())



                    timestamp      id       group action
0  2016-09-24 17:42:27.839496  804196  experiment   view
1  2016-09-24 19:19:03.542569  434745  experiment   view
2  2016-09-24 19:36:00.944135  507599  experiment   view
3  2016-09-24 19:59:02.646620  671993     control   view
4  2016-09-24 20:26:14.466886  536734  experiment   view


## Conduct a Statistical Test

Conduct a statistical test to determine whether the experimental homepage was more effective than that of the control group.

In [5]:
grouped_data = df.groupby(['group', 'action'])['id'].count().unstack()
grouped_data['total'] = grouped_data.sum(axis=1)
grouped_data['conversion_rate'] = grouped_data['view'] / grouped_data['total']

experimental_conversion_rate = grouped_data.loc['experiment', 'conversion_rate']
control_conversion_rate = grouped_data.loc['control', 'conversion_rate']

print("Experimental Conversion Rate:", experimental_conversion_rate)
print("Control Conversion Rate:", control_conversion_rate)

from statsmodels.stats.proportion import proportions_ztest

#view' actions for each group
experimental_views = grouped_data.loc['experiment', 'view']
control_views = grouped_data.loc['control', 'view']

#total counts for each group
experimental_total = grouped_data.loc['experiment', 'total']
control_total = grouped_data.loc['control', 'total']

# Z-test
count = [experimental_views, control_views]
nobs = [experimental_total, control_total]
stat, pval = proportions_ztest(count, nobs)

print("Z-score:", stat)
print("P-value:", pval)


Experimental Conversion Rate: 0.763506625891947
Control Conversion Rate: 0.7814258911819888
Z-score: -1.9332699751308169
P-value: 0.0532029485228941


## Verifying Results

One sensible formulation of the data to answer the hypothesis test above would be to create a binary variable representing each individual in the experiment and control group. This binary variable would represent whether or not that individual clicked on the homepage; 1 for they did and 0 if they did not. 

The variance for the number of successes in a sample of a binomial variable with n observations is given by:

## $n\bullet p (1-p)$

Given this, perform 3 steps to verify the results of your statistical test:
1. Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 
2. Calculate the number of standard deviations that the actual number of clicks was from this estimate. 
3. Finally, calculate a p-value using the normal distribution based on this z-score.

### Step 1:
Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 

In [6]:


# Calculate the expected number of clicks for the experiment group
expected_clicks_experiment = experimental_conversion_rate * experimental_total

print("Expected Clicks for the Experiment Group:", expected_clicks_experiment)


Expected Clicks for the Experiment Group: 2996.0


### Step 2:
Calculate the number of standard deviations that the actual number of clicks was from this estimate.

In [11]:
import math


actual_clicks_experiment = 123   # Replace 123 with the actual number of clicks

# Assuming you have the expected number of clicks for the experiment group as expected_clicks_experiment (calculated in Step 1)
expected_clicks_experiment = 2996.0  # Replace 2996.0 with the actual value from your calculation in Step 1

# Assuming you have the conversion rate for the experiment group as experimental_conversion_rate
experimental_conversion_rate = 0.1   # Replace 0.1 with the actual conversion rate for the experiment group

# Calculate the number of standard deviations from the estimate
Z_score = (actual_clicks_experiment - expected_clicks_experiment) / math.sqrt(expected_clicks_experiment * (1 - experimental_conversion_rate))

print("Z-score:", Z_score)





Z-score: -55.327808288190674


### Step 3: 
Finally, calculate a p-value using the normal distribution based on this z-score.

In [12]:
import scipy.stats as stats

# Assuming you have already calculated the Z-score as Z_score

# Calculate the two-tailed p-value
p_value = 2 * (1 - stats.norm.cdf(abs(Z_score)))

print("p-value:", p_value)


p-value: 0.0


### Analysis:

Does this result roughly match that of the previous statistical test?

> Comment: *Having a p-value of <.05 there is strong evidence to reject the null hypothesis. This indicates that the experimental group had a higher click rate than the control gorup.**

## Summary

In this lab, you continued to get more practice designing and conducting AB tests. This required additional work preprocessing and formulating the initial problem in a suitable manner. Additionally, you also saw how to verify results, strengthening your knowledge of binomial variables, and reviewing initial statistical concepts of the central limit theorem, standard deviation, z-scores, and their accompanying p-values.